Notebook to process the issuances from different registries.

In [1]:
import numpy as np
import pandas as pd

In [25]:
# BC Carbon Registry Issuances
root_BCregistry = '../data/offset_registries/BCcarbon_registry/'
bc_issuances = pd.read_csv(f'{root_BCregistry}BCcarbon_registry_issuances.csv')
bc_issuances = bc_issuances.rename(columns = {'Project':'Project Name', 'Units': 'Issuances'})
# Remove , in Issuances
bc_issuances['Issuances'] = bc_issuances['Issuances'].str.replace(',', '').astype(float)

In [3]:
# UBerk Issuances excel
uberk_raw = pd.read_excel('../data/offset_registries/UBerkley/UBerk_Voluntary-Registry-Offsets-Database--v2024-12-year-end.xlsx', 
                          sheet_name='PROJECTS', skiprows=3)

In [4]:
# take first column and columns 12 to 52
uberk_issuances = uberk_raw.iloc[:, [0] + list(range(23,52))]

# Pivot year columns to long format
uberk_issuances = uberk_issuances.melt(id_vars='Project ID', var_name='Vintage', value_name='Issuances')

# Join the rest of the metadata
uberk_issuances = uberk_issuances.merge(uberk_raw.iloc[:, [0, 1, 5, 6, 11]], on='Project ID')

# Filter for Canada
uberk_issuances = uberk_issuances[uberk_issuances['Country'] == 'Canada'].drop(columns=['Country']).rename(columns={'Scope': 'Project Type'})

In [5]:
print(bc_issuances.columns)
print(uberk_issuances.columns)

Index(['Vintage', 'Project Name', 'Account', 'Project Type', 'Verifier',
       'Issuances', 'Measurement', 'Details'],
      dtype='object')
Index(['Project ID', 'Vintage', 'Issuances', 'Project Name', 'Project Type',
       'Type'],
      dtype='object')


In [35]:
# Combine the two datasets 
issuances = pd.concat([bc_issuances[['Vintage', 'Project Name', 'Issuances', 'Project Type']], 
                       uberk_issuances[['Vintage', 'Project Name', 'Issuances', 'Project Type', 'Type']]], ignore_index=True, axis=0)
# Calculate the total issuances by project
issuances['Issuances'] = pd.to_numeric(issuances['Issuances'], errors='coerce')
total_issuances = issuances[['Project Name', 'Project Type', 'Issuances']].groupby(['Project Name', 'Project Type']).sum().reset_index().rename(columns={'Issuances': 'Total Issuances'})

issuances = issuances.merge(total_issuances, on=['Project Name', 'Project Type'], how='left').reset_index()

## TODO: Deal with vintages that span multiple years

forest_issuances = issuances[(issuances['Project Type'] == 'Sequestration') | (issuances['Project Type'] == 'Forestry & Land Use')]
forest_issuances.to_csv('../data/offset_registries/forestry_projects_issuances.csv', index=False)